In [82]:
import pandas as pd
import requests
import time
import math
from tqdm import tqdm

In [83]:
file_url = "./make_file/공동작업/서울열선_광진구_필터링파일.csv"
file_encoding = "UTF-8"

In [84]:
data = pd.read_csv(file_url, encoding=file_encoding, low_memory=False)

In [85]:
try :
    data = data.drop(columns="id")
    print(f"data.drop(columns=\"id\")  완료.")
except KeyError as e :
    print(e)

data.drop(columns="id")  완료.


In [86]:
data.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역'],
      dtype='object')

In [87]:
print(data.isnull().sum())

도로명         0
열선          0
도로 종류       2
시작점_위도      0
시작점_경도      0
종료점_위도      0
종료점_경도      0
중앙점_위도      0
중앙점_경도      0
도로_길이       0
행정동         0
도로폭       154
도로규모      154
행정구역        0
dtype: int64


In [88]:
start_latitude = data['시작점_위도'].to_list()
start_longitude = data['시작점_경도'].to_list()

end_latitude = data['종료점_위도'].to_list()
end_longitude = data['종료점_경도'].to_list()

In [89]:
print(len(start_latitude))
print(len(start_longitude))
print(len(end_latitude))
print(len(end_longitude))

7193
7193
7193
7193


### 구글

In [90]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
print(open_api_key_file['사이트'].value_counts())
GOOGLE_API_KEY = open_api_key_file[open_api_key_file['사이트'] == 'GOOGLE_MAP'].values[0][1]   

사이트
공공데이터                     1
kakao_api_key_REST        1
kakao_api_key_REST_sub    1
카카오 JS                    1
국토부v-world                1
GOOGLE_MAP                1
행정안전부                     1
Name: count, dtype: int64


In [91]:
start_altitude = []
end_altitude = []

In [92]:
def google_api(latitudes, longitudes, mode):
    global start_altitude, end_altitude
    
    url = f"https://maps.googleapis.com/maps/api/elevation/json?key={GOOGLE_API_KEY}"

    locations = list(zip(latitudes, longitudes))  # (위도, 경도) 쌍 리스트
    
    for i in range(0, len(locations), 400):  # 최대 512개씩 요청
        batch_locations = locations[i:i+400]  # 최대 512개씩 묶기
        locations_param = '|'.join([f"{lat},{lon}" for lat, lon in batch_locations])
        full_url = f"{url}&locations={locations_param}"
        
        response = requests.get(full_url)
            
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'OK':
                elevations = [result['elevation'] for result in data['results']]   
            
                if mode == 'start':
                    start_altitude.extend(elevations)
                elif mode == 'end':
                    end_altitude.extend(elevations)
            else:
                print("Error in response:", data['status'])
        else:
            print("API 호출 실패:", response.status_code)

        time.sleep(0.1)


In [93]:
print(f'전) 기점 고도 길이 : {len(start_altitude)}') 

print(f'전) 종점 고도 길이 : {len(end_altitude)}') 

전) 기점 고도 길이 : 0
전) 종점 고도 길이 : 0


In [94]:
# API 호출 (start)
print(f'기점의 고도값 불러오기')
print(f'시작') 
google_api(start_latitude, start_longitude, 'start') 
print(f'종료')

기점의 고도값 불러오기
시작
종료


In [95]:
# API 호출 (end)
print(f'종점의 고도값 불러오기')
print(f'시작')
google_api(end_latitude, end_longitude, 'end')
print(f'종료')

종점의 고도값 불러오기
시작
API 호출 실패: 400
종료


In [96]:
print(f'후) 기점 고도 길이 : {len(start_altitude)}') 
print(f'후) 종점 고도 길이 : {len(end_altitude)}') 

if (len(start_altitude) == len(end_altitude)) and (len(start_altitude) > 0) :
    print(f'결과의 길이가 같습니다.')
    print(f'성공적으로 처리되었습니다.')
else :
    print(f'데이터 처리에 실패했습니다...')

후) 기점 고도 길이 : 7193
후) 종점 고도 길이 : 6793
데이터 처리에 실패했습니다...


In [97]:
altitude_difference = []

for i in range(len(start_altitude)) :
    result = abs(start_altitude[i] - end_altitude[i]) 
    altitude_difference.append(result)

print(len(altitude_difference))

IndexError: list index out of range

In [ ]:
# 결과 데이터 추가 및 저장
data['시작점_고도'] = start_altitude
data['종료점_고도'] = end_altitude
data['고도의_차이'] = altitude_difference

In [ ]:
start_lat = data['시작점_위도'].to_list()
start_lon = data['시작점_경도'].to_list()
end_lat = data['종료점_위도'].to_list()
end_lon = data['종료점_경도'].to_list()
different_alt = data['고도의_차이'].to_list()

In [ ]:
print(f'start_lat 길이 : {len(start_lat)}')
print(f'start_lon 길이 : {len(start_lon)}')
print(f'end_lat 길이 : {len(end_lat)}')
print(f'end_lon 길이 : {len(end_lon)}')
print(f'different_alt 길이 : {len(different_alt)}')

In [ ]:
# 지구 반지름 (미터)
R = 6371000   

In [ ]:
# 결과 저장 리스트
road_slope = []

In [ ]:
for i in range(len(start_lat)):
    # 위경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(math.radians, [start_lat[i], start_lon[i], end_lat[i], end_lon[i]])

    # 위도, 경도의 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine 공식으로 수평 거리(d) 계산
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c  # 두 지점 간 수평 거리 (m)

    # 고도 차이 (h)
    h = different_alt[i]  

    # 경사각(θ) 계산 (라디안 → 도 변환)
    theta = math.degrees(math.atan2(h, d))

    # 결과 저장
    road_slope.append(theta)

    if(i < 5) :
        # 디버깅 출력
        print(f"Point {i + 1}: d = {d:.4f} m, h = {h:.4f} m, θ = {theta:.4f}°")


In [ ]:
print(f'변환한 road_slope의 길이 : {len(road_slope)}')

In [ ]:
new_add_df = pd.DataFrame({
    '경사각' : road_slope
})

In [ ]:
data['경사각'] = new_add_df

In [ ]:
data['도로 종류'].unique()

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
print(f"최대 경사도 : {data['경사각'].max()}")
print(f"평균 경사도 : {data['경사각'].mean()}")
print(f"최소 경사도 : {data['경사각'].min()}")

In [ ]:
try :
    save_data_url = './make_file/(경사도)_서울열선_광진도로.csv'
    data.to_csv(save_data_url, encoding='UTF-8')
    print(f"{save_data_url} 저장 완료")
except :
    print("err") 

In [ ]:
data.columns